# Discovery

Required packages: vfb-connect and python-catmaid (pymaid & navis)

In [ ]:
!pip install vfb-connect --upgrade
!pip install python-catmaid --upgrade

Requirement already up-to-date: vfb-connect in /root/venv/lib/python3.7/site-packages (1.2.5)
^C
Traceback (most recent call last):
  File "/root/venv/bin/pip", line 8, in <module>
    sys.exit(main())
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/main.py", line 75, in main
    return command.main(cmd_args)
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/base_command.py", line 114, in main
    return self._main(args)
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/base_command.py", line 226, in _main
    self.handle_pip_version_check(options)
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/req_command.py", line 155, in handle_pip_version_check
    timeout=min(5, options.timeout)
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/req_command.py", line 100, in _build_session
    index_urls=self._get_index_urls(options),
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/network/session.py", line

### How to find neurons across datasets

VirtualFlyBrain integrates images and connectomics profiles of neurons from many sources.  It classifies and records their properties using a standard, queryable classification ([The Drosophila Anatomy Ontology](https://www.ebi.ac.uk/ols/ontologies/fbbt)). This standardises the names of neuron types across sources, so you don't need to worry about differences in nomenclature uses and supports queries for neurons by their classification. 

In [ ]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd
vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
Server is running CATMAID version 2020.02.15-905-g93a969b37


### Finds neurons by type (classification) across datasets

We can use the `vc.get_instances` method in combination with the name of a neuron type on VFB to find individual neurons from multiple sources.

Use the search tool on [VFB](http://virtualflybrain.org) to find neuron types by name or synonym:

<img src="https://user-images.githubusercontent.com/112839/109564687-535c9380-7ad9-11eb-80e9-5a5bc21cd915.png" width=30% height=30%>
<img src="https://user-images.githubusercontent.com/112839/109565128-eac1e680-7ad9-11eb-9649-1ec55a298a1b.png" width=30% height=30%>

Use either the full name or the Symbol to query for neurons:

In [ ]:
DA3adPN = vc.get_instances("adult Drosulfakinin neuron", summary=True)
pd.DataFrame.from_records(DA3adPN)

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,DSKMP3_R - 327937328,,VFB_jrchjti6,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,327937328,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,DSKMP1A(PVM02)_L - 1260833150,,VFB_jrchjti3,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1260833150,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,DSKMP3_R - 328559607,,VFB_jrchjti7,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,328559607,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,DSKMP1B_R - 1352077058,,VFB_jrchjti4,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1352077058,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
4,DSKMP1B(PVM02)_L - 1011184205,,VFB_jrchjti5,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1011184205,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
5,DSKMP1A_R - 1135837629,,VFB_jrchjti2,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1135837629,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


### Find neurons by location

We can use the same method to search for neurons by location, using simple queries.

In [ ]:
# Find neurons by location. The following query works across multiple data sources and both sides of the brain.  
# Results may be incomplete & may include minor overlap inferred from low synapse counts

neurons_in_DA3 = vc.get_instances("'neuron' that 'overlaps' some 'antennal lobe glomerulus DA3'", summary=True)
neurons_in_DA3_tab = pd.DataFrame.from_records(neurons_in_DA3)
neurons_in_DA3_tab[0:5]


Running query: FBbt:00005106 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00005106+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 158


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,lLN2T_a(Tortuous)_R - 5813056598,,VFB_jrchk8bi,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,5813056598,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,ORN_DL3_R - 1671625186,,VFB_jrchk1hj,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult olfactory receptor neuron Or65,FBbt_00067011,neuprint_JRC_Hemibrain_1point1,1671625186,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,DA4m_adPN_R - 574037266,,VFB_jrchjtdq,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA4m adPN,FBbt_00047714,neuprint_JRC_Hemibrain_1point1,574037266,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,Uniglomerular mALT DA3 adPN#L1 (FAFB:2449792),,VFB_0010123h,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe projection neuron DA3 adPN,FBbt_00100384,catmaid_fafb,2449792,JRC2018Unisex,BatesSchlegel2020,https://creativecommons.org/licenses/by-sa/4.0...
4,lLN2T_e(Tortuous)_R - 1699974843,,VFB_jrchk8br,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1699974843,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


In [ ]:
# Find local interneurons (intrinsic neurons) of the AL, overlapping DA3:

local_in_DA3 = vc.get_instances("'local interneuron of adult antennal lobe' that 'overlaps' some 'antennal lobe glomerulus DA3'",
                                summary=True)
local_in_DA3_tab = pd.DataFrame.from_records(local_in_DA3)
local_in_DA3_tab

Running query: FBbt:00007390 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00007390+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 53


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,lLN16b_R - 1887168462,,VFB_jrchk89w,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,1887168462,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,lLN1_c_R - 5813047691,,VFB_jrchk8ae,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,5813047691,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,lLN2S(Star)_R - 1670627928,,VFB_jrchk8bb,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2S,FBbt_00049815,neuprint_JRC_Hemibrain_1point1,1670627928,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,lLN2F_b(Full)_R - 5813024698,,VFB_jrchk8an,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2F,FBbt_00049812,neuprint_JRC_Hemibrain_1point1,5813024698,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
4,lLN2T_c(Tortuous)_R - 1671292719,,VFB_jrchk8bo,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1671292719,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
5,lLN1_c_R - 5813062199,,VFB_jrchk8af,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,5813062199,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
6,lLN1_a_R - 5813130064,,VFB_jrchk8a4,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,5813130064,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
7,lLN1_c_R - 1702651358,,VFB_jrchk8aj,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,1702651358,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
8,lLN2T_e(Tortuous)_R - 1640922516,,VFB_jrchk8bs,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult antennal lobe lateral local neuron 2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1640922516,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
9,lLN11_R - 1670278227,,VFB_jrchk89e,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult local interneuron of the lateral ALl1 ne...,FBbt_00007394,neuprint_JRC_Hemibrain_1point1,1670278227,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


In [ ]:
# Find neurons by dataset/paper - on CATMAID

bates = pymaid.find_neurons(annotations='Paper: Bates and Schlegel et al 2020')
bates

INFO  : Found 583 neurons matching the search parameters (pymaid)


,type,name,skeleton_id,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,CatmaidNeuron,Uniglomerular mALT DA1 lPN 57316 2863105 ML,2863104,NA,NA,NA,NA,NA,NA,1 nanometer
1,CatmaidNeuron,Uniglomerular mALT DA3 adPN 57350 HG,57349,NA,NA,NA,NA,NA,NA,1 nanometer
...,...,...,...,...,...,...,...,...,...,...
581,CatmaidNeuron,Multiglomerular mlALT vPN VM7d+VM5d+DC4+6 LTS ...,4624378,NA,NA,NA,NA,NA,NA,1 nanometer
582,CatmaidNeuron,Uniglomerular mALT DL2d adPN 57342 ML,57341,NA,NA,NA,NA,NA,NA,1 nanometer


In [ ]:
# Inspect what datasets are available on VFB

ds = vc.neo_query_wrapper.get_datasets(summary=True)
ds_tab = pd.DataFrame.from_records(ds)
ds_tab.sort_values(by=['id'])

,label,symbol,id,tags,description,miniref,FlyBase,PMID,DOI
81,EM L1 Andrade et al. 2019,,Andrade2019,Entity|Individual|DataSet,[L1 EM reconstructed neurons from Andrade et a...,"Andrade et al., 2019, Curr. Biol. 29(3): 412--...",FBrf0241389,30661802,10.1016/j.cub.2018.12.012
16,MBONs and split-GAL4 lines that target them (A...,,Aso2014,Entity|Individual|DataSet,[],"Aso et al., 2014, eLife 3: e04577",FBrf0227179,25535793,10.7554/eLife.04577
19,split-GAL4 lines for dopaminergic neurons (Aso...,,AsoRubin2016,Entity|Individual|DataSet,[For comparison of the properties of memories ...,"Aso and Rubin, 2016, eLife 5: e16135",FBrf0233230,27441388,10.7554/eLife.16135
52,EM FAFB Bates and Schlegel et al 2020,,BatesSchlegel2020,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Bates and ...,"Bates and Schlegel et al., 2020",,,10.1016/j.cub.2020.06.042
0,Larval olfactory system neurons - EM (Berk2016),,Berck2016,Entity|Individual|DataSet,"[Berck, Khandelwal et al. 2016]","Berck et al., 2016, eLife 5: e14859",FBrf0232785,27177418,10.7554/eLife.14859
...,...,...,...,...,...,...,...,...,...
30,Lee lab adult brain lineage clone image set,,Yu2013,Entity|Individual|DataSet,[An exhaustive set of lineage clones covering ...,"Yu et al., 2013, Curr. Biol. 23(8): 633--643",FBrf0221412,23541733,10.1016/j.cub.2013.02.057
15,"EM L1 Zarin, Mark et al. 2019",,Zarin2019,Entity|Individual|DataSet,"[L1 EM reconstructed neurons from Zarin, Mark ...","Zarin, Mark et al., 2019",,,10.1101/617977
48,EM FAFB Zheng et al 2018,,Zheng2018,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Zheng et a...,"Zheng et al., 2018, Cell 174(3): 730--743.e22",FBrf0239557,30033368,10.1016/j.cell.2018.06.019
34,EM FAFB Zheng et al 2020,,Zheng2020,Entity|Individual|DataSet,[FAFB EM reconstructed neurons from Zheng et a...,Zheng et al. 2020,,,10.1101/2020.04.17.047167


In [ ]:
sayin_tab = pd.DataFrame.from_records(vc.get_instances_by_dataset('Sayin2019', summary=True))
sayin_tab

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,OA-VPM3 (FAFB:1329078),,VFB_001001dr,Entity|Octopaminergic|Adult|Anatomy|has_image|...,octopaminergic VPM3 neuron,FBbt_00110151,catmaid_fafb,1329078,JRC2018Unisex,Sayin2019,https://creativecommons.org/licenses/by-sa/4.0...
1,OA-VPM4 (FAFB:1191261),,VFB_001001dq,Entity|Octopaminergic|Adult|Anatomy|has_image|...,octopaminergic VPM4 neuron,FBbt_00110152,catmaid_fafb,1191261,JRC2018Unisex,Sayin2019,https://creativecommons.org/licenses/by-sa/4.0...


In [ ]:
vc.get_connected_neurons_by_type(upstream_type='LNd', downstream_type='adult descending neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjzxx,LNd_R - 5813021192,25,VFB_jrchjthm,DNp27_R - 1072063538,LNd neuron,descending neuron of the posterior brain DNp27,neuprint_JRC_Hemibrain_1point1,5813021192,neuprint_JRC_Hemibrain_1point1,1072063538


In [ ]:
# Intra pacemaker neuron neuron connections

vc.get_connected_neurons_by_type(upstream_type='pacemaker neuron', downstream_type='pacemaker neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjzxy,LNd_R - 5813069648,37,VFB_jrchk8e0,5th s-LNv - 511051477,LNd neuron,s-LNv neuron,neuprint_JRC_Hemibrain_1point1,5813069648,neuprint_JRC_Hemibrain_1point1,511051477
1,VFB_jrchk089,LPN_R - 480029788,43,VFB_jrchk08a,LPN_R - 450034902,LP neuron,LP neuron,neuprint_JRC_Hemibrain_1point1,480029788,neuprint_JRC_Hemibrain_1point1,450034902
2,VFB_jrchjtf0,DN1pA_R - 324846570,25,VFB_jrchk8e0,5th s-LNv - 511051477,DN1p neuron,s-LNv neuron,neuprint_JRC_Hemibrain_1point1,324846570,neuprint_JRC_Hemibrain_1point1,511051477
3,VFB_jrchjtf0,DN1pA_R - 324846570,37,VFB_jrchjzxy,LNd_R - 5813069648,DN1p neuron,LNd neuron,neuprint_JRC_Hemibrain_1point1,324846570,neuprint_JRC_Hemibrain_1point1,5813069648
4,VFB_jrchk8e0,5th s-LNv - 511051477,40,VFB_jrchjzxy,LNd_R - 5813069648,s-LNv neuron,LNd neuron,neuprint_JRC_Hemibrain_1point1,511051477,neuprint_JRC_Hemibrain_1point1,5813069648
5,VFB_jrchjtf3,DN1pA_R - 387166379,25,VFB_jrchk8e0,5th s-LNv - 511051477,DN1p neuron,s-LNv neuron,neuprint_JRC_Hemibrain_1point1,387166379,neuprint_JRC_Hemibrain_1point1,511051477
6,VFB_jrchjtf1,DN1pA_R - 325529237,30,VFB_jrchjzxy,LNd_R - 5813069648,DN1p neuron,LNd neuron,neuprint_JRC_Hemibrain_1point1,325529237,neuprint_JRC_Hemibrain_1point1,5813069648
7,VFB_jrchk8e0,5th s-LNv - 511051477,25,VFB_jrchjzxx,LNd_R - 5813021192,s-LNv neuron,LNd neuron,neuprint_JRC_Hemibrain_1point1,511051477,neuprint_JRC_Hemibrain_1point1,5813021192
8,VFB_jrchjtez,DN1pA_R - 5813010153,25,VFB_jrchk8e0,5th s-LNv - 511051477,DN1p neuron,s-LNv neuron,neuprint_JRC_Hemibrain_1point1,5813010153,neuprint_JRC_Hemibrain_1point1,511051477
9,VFB_jrchjzxy,LNd_R - 5813069648,29,VFB_jrchjzxw,LNd_R - 5813056917,LNd neuron,LNd neuron,neuprint_JRC_Hemibrain_1point1,5813069648,neuprint_JRC_Hemibrain_1point1,5813056917


In [ ]:

vc.get_connected_neurons_by_type(upstream_type='adult neuron', downstream_type='adult Drosulfakinin neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjsj3,AstA1_L - 362473525,28,VFB_jrchjti4,DSKMP1B_R - 1352077058,adult neuron,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,362473525,neuprint_JRC_Hemibrain_1point1,1352077058
1,VFB_jrchk5zh,SLP384_R - 482702027,20,VFB_jrchjti6,DSKMP3_R - 327937328,adult superior lateral protocerebrum neuron 384,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,482702027,neuprint_JRC_Hemibrain_1point1,327937328
2,VFB_jrchjrjn,AVLP001_R - 1321564092,26,VFB_jrchjti2,DSKMP1A_R - 1135837629,adult anterior ventrolateral protocerebrum neu...,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,1321564092,neuprint_JRC_Hemibrain_1point1,1135837629
3,VFB_jrchk5eo,SLP066_R - 327843160,42,VFB_jrchjti4,DSKMP1B_R - 1352077058,adult superior lateral protocerebrum neuron 066,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,327843160,neuprint_JRC_Hemibrain_1point1,1352077058
4,VFB_jrchk5qf,SLP244_R - 298214577,27,VFB_jrchjti6,DSKMP3_R - 327937328,adult superior lateral protocerebrum neuron 244,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,298214577,neuprint_JRC_Hemibrain_1point1,327937328
...,...,...,...,...,...,...,...,...,...,...,...
56,VFB_jrchk6zu,SMP449_R - 5813026592,25,VFB_jrchjti4,DSKMP1B_R - 1352077058,adult superior medial protocerebrum neuron 449,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,5813026592,neuprint_JRC_Hemibrain_1point1,1352077058
57,VFB_jrchk5jv,SLP152_R - 487812620,28,VFB_jrchjti4,DSKMP1B_R - 1352077058,adult superior lateral protocerebrum neuron 152,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,487812620,neuprint_JRC_Hemibrain_1point1,1352077058
58,VFB_jrchjrxe,AVLP223_R - 1356743512,27,VFB_jrchjti2,DSKMP1A_R - 1135837629,adult anterior ventrolateral protocerebrum neu...,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,1356743512,neuprint_JRC_Hemibrain_1point1,1135837629
59,VFB_jrchk5qg,SLP244_R - 297533212,33,VFB_jrchjti6,DSKMP3_R - 327937328,adult superior lateral protocerebrum neuron 244,adult Drosulfakinin neuron,neuprint_JRC_Hemibrain_1point1,297533212,neuprint_JRC_Hemibrain_1point1,327937328
